### Setting.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 14.2 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 6.8 MB 56.7 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd 
import numpy as np 
import os
import torch
import torch.nn as nn

import warnings 
warnings.filterwarnings("ignore")
from tqdm import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer, AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
import re
from sklearn.model_selection import train_test_split

In [ ]:
# Random Seed Fix
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

In [ ]:
def load_data(path):
  TRAIN = os.path.join(path, 'benchmark_train_data.csv')
  AUG = os.path.join(path, 'aug_train_data.csv')
  TEST = os.path.join(path, 'test_data.csv')
  SS = os.path.join(path, 'sample_submission.csv')
  label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}
  train = pd.read_csv(TRAIN)
  aug = pd.read_csv(AUG)[:10000]
  
  test = pd.read_csv(TEST)
  sample_submission = pd.read_csv(SS)
  train['label'] = train['label'].map(label_dict)
  aug['label'] = aug['label'].map(label_dict)
  
  train = train.append(aug)
  train.reset_index(inplace=True)
  #train.set_index()
  display(train)
  return train,test,sample_submission

def text_clean(df):
  #=df["premise_"] = "<s>" + df["premise"].astype(str) + "[SEP]"
  df["premise_"] = "[CLS]"+df["premise"].astype(str)
  #df["hypothesis_"] = df["hypothesis"].astype(str) + "[SEP]"
  df["hypothesis_"] = df["hypothesis"].astype(str) + "[SEP]"
  df["text_sum"] = df.premise_ + "[SEP]" + df.hypothesis_
  df = df[['text_sum','label']]
  return df 

def random_deletion(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence)

def eda_aug(df):

    cache = {'premise':[], 'hypothesis':[], 'label':[]}
    for idx in tqdm(range(len(df))):
        premise = df.iloc[idx]['premise']
        hypothesis = df.iloc[idx]['hypothesis']
        label = df.iloc[idx]['label']
        cache['premise'].append(premise)
        cache['hypothesis'].append(hypothesis)
        cache['label'].append(label)
        flag = random.randrange(10)
        if flag < 2:
          cache['premise'].append(random_deletion(premise))
          cache['hypothesis'].append(random_deletion(hypothesis))
          cache['label'].append(label)
          cache['premise'].append(random_swap(premise))
          cache['hypothesis'].append(random_swap(hypothesis))
          cache['label'].append(label)
    
    return pd.DataFrame(cache)

### Dataset.py

In [ ]:
#ROOT = '/content/drive/MyDrive/DACON_MONTHLYNLI'
DATA = '/content/drive/Shareddrives/Dacon/data'
train,test,sample_submission = load_data(DATA)
###### AUGMENTATION ######
#train = eda_aug(train)
###### AUGMENTATION ######

clean_train,clean_test  = text_clean(train),text_clean(test)
display(clean_train)

,level_0,index,premise,hypothesis,label
0,0,0.0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,1
1,1,1.0,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,1
2,2,2.0,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,0
3,3,3.0,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,2
4,4,4.0,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,2
...,...,...,...,...,...
37993,9995,9995.0,2000년 집권한 로랑 그바그보 대통령은 지난해 11월 대선에서 패배해 재선에 실패했다.,재선에 실패한 대통령은 다시 대통령에 출마할 수 없다.,2
37994,9996,9996.0,시부야와 롯데로 이사하는 것이 좋습니다.,시부야나 롯데로만 이동할 수 있어요.,2
37995,9997,9997.0,"다만 지리산, 설악산, 덕유산 등 전국 국립공원 내 쉼터 14곳은 탐방객의 안전과 ...",전국 14개 국립공원이 모두 개장 대상에 포함됐다.,1
37996,9998,9998.0,이것은 현대 세계에서 고군분투하는 한 청소년에 관한 영화입니다.,그것은 현대 사회의 청소년들에 관한 영화였습니다.,0


,text_sum,label
0,"[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나...",1
1,[CLS]삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 ...,1
2,[CLS]이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.[SEP]예측적 범...,0
3,[CLS]광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민...,2
4,"[CLS]진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 ...",2
...,...,...
37993,[CLS]2000년 집권한 로랑 그바그보 대통령은 지난해 11월 대선에서 패배해 재...,2
37994,[CLS]시부야와 롯데로 이사하는 것이 좋습니다.[SEP]시부야나 롯데로만 이동할 ...,2
37995,"[CLS]다만 지리산, 설악산, 덕유산 등 전국 국립공원 내 쉼터 14곳은 탐방객의...",1
37996,[CLS]이것은 현대 세계에서 고군분투하는 한 청소년에 관한 영화입니다.[SEP]그...,0


In [ ]:
model_roberta = AutoModel.from_pretrained("klue/roberta-large")
tokenizer_roberta = AutoTokenizer.from_pretrained("klue/roberta-large")

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def roberta_transform(text):
  transform = tokenizer_roberta(text,
                                pad_to_max_length=True,
                               truncation=True,
                               max_length=256,
                               return_tensors='pt',
                                add_special_tokens=False)
  return transform

In [ ]:
class customDataset(Dataset):
  def __init__(self,dataset,mode='train',transform=roberta_transform):
    super(customDataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform

  def __getitem__(self, idx):
    
    text = self.dataset['text_sum'][idx]
    tokens = self.transform(text)
    token_ids = tokens['input_ids'][0]  # tensor of token ids
    attn_masks = tokens['attention_mask'][0]  # binary tensor with "0" for padded values and "1" for the other values
    token_type_ids = tokens['token_type_ids'][0]  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    if self.mode == 'test':
      return token_ids,attn_masks,token_type_ids
    else: 
      labels = self.dataset['label'][idx]
      return token_ids,attn_masks,token_type_ids, labels
  
  def __len__(self):
    return(len(self.dataset))

### Model.py

In [ ]:
class ROBERTaClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 1024,
                 num_classes=3,   ##클래스 수 조정##
                 params=None,
                 freeze_bert=False):
        super(ROBERTaClassifier, self).__init__()
        self.bert = bert
        self.freeze_bert=freeze_bert

        if self.freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

                 
        self.classifier = nn.Linear(hidden_size , 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc_layer1 = nn.Linear(256,128)
        self.fc_layer2 = nn.Linear(128,num_classes)
    

    def forward(self, input_ids, attn_masks):
        
        _,pooler = self.bert(input_ids, attn_masks, return_dict=False)
        output1 = self.classifier(pooler)
        output2 = self.fc_layer1(output1)
        output3 = self.fc_layer2(self.dropout(output2))
        return (output3)

### Train.py

In [ ]:
device = torch.device("cuda")
!nvidia-smi

Sun Feb 27 00:19:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
###### HYPERPARMS ######
lr = 2e-5

batch_size=8
warmup_ratio = 0.06
num_epochs = 10

log_interval = 500
#learning_rate = 5e-6

In [ ]:
############### CV ################
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5,shuffle=True,random_state=42)
folds=[]
for trn_idx,val_idx in skf.split(clean_train['text_sum'], train['label']):
    folds.append((trn_idx,val_idx))

In [ ]:


############### CV Training ###############
best_models = []


for i,fold in enumerate(range(4,5)):
    print('===============',i+1,'fold start===============')
    model = ROBERTaClassifier(model_roberta).to(device)
    model=nn.DataParallel(model).to(device)
    
    optimizer = AdamW(model.parameters(), lr=lr)

    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    train_data = train.loc[train_idx]
    val_data = train.loc[valid_idx]
    train_dataset = customDataset(train_data,'train')
    valid_dataset = customDataset(val_data,'train')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    total_steps = len(train_loader) * num_epochs
    warmup_step = int(total_steps * warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=5, num_training_steps=total_steps)

    
    for epoch in range( num_epochs):
        batches = 0
        total_loss = 0.0
        correct = 0
        total =0
        model.train()
        
        for input_ids_batch, attention_masks_batch, token_type_ids_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device),attn_masks= attention_masks_batch.to(device))
            loss1 = F.cross_entropy(y_pred, y_batch)
            one_hot = torch.nn.functional.one_hot(y_batch, 3).to(torch.float32)
            loss2 = F.mse_loss(y_pred, one_hot)
            loss3= F.hinge_embedding_loss(y_pred, one_hot)
            loss = 0.7*loss1+0.15*loss2+0.15*loss3
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)
            batches += 1
            if batches % log_interval == 0:
                print("Batch Loss: ", total_loss / batches, "Accuracy: ", correct.float() / total)
        
        val_loss = []
        val_acc = []
        
        for input_ids_batch, attention_masks_batch, token_type_ids, y_batch in tqdm(valid_loader):
            
            model.eval()
            with torch.no_grad():
                
                y_pred = model(input_ids_batch.to(device),attn_masks= attention_masks_batch.to(device))
                valid_loss = F.cross_entropy(y_pred,y_batch.to(device)).cpu().detach().numpy()

                preds = torch.argmax(y_pred,1)
                preds = preds.cpu().detach().numpy()
                y_batch = y_batch.cpu().detach().numpy()
                batch_acc = (preds==y_batch).mean()
                val_loss.append(valid_loss)
                val_acc.append(batch_acc)
                
        
        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)
        
        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
        print(optimizer.param_groups[0]["lr"])
        if epoch % 3 == 0:
            valid_acc_max = val_acc
            best_models.append(model)
            torch.save(model.state_dict(), f'/content/drive/Shareddrives/Dacon/hongsun/ROBERTa_large_fold_{fold}_{epoch}.pth') 
            print('model save, model val acc : ',val_acc)
            print('best_models size : ',len(best_models))

=============== 1 fold start===============


 13%|█▎        | 500/3800 [06:12<41:13,  1.33it/s]

Batch Loss:  0.9216977579593658 Accuracy:  tensor(0.3473, device='cuda:0')


 26%|██▋       | 1000/3800 [12:25<34:47,  1.34it/s]

Batch Loss:  0.9198011909723282 Accuracy:  tensor(0.3544, device='cuda:0')


...

Batch Loss:  0.914828966041406 Accuracy:  tensor(0.3427, device='cuda:0')


 92%|█████████▏| 3500/3800 [43:29<03:43,  1.34it/s]

Batch Loss:  0.9149523602553776 Accuracy:  tensor(0.3417, device='cuda:0')


100%|██████████| 950/950 [04:00<00:00,  3.95it/s]


Epoch: 9 - valid Loss: 1.098189 - valid_acc : 0.342538
0.0
model save, model val acc :  0.3425375939849624
best_models size :  4
